In [1]:
from numpy import mean
from numpy import std
from numpy import dstack
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import LSTM
from tensorflow.keras.regularizers import l2
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf


In [2]:
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only use the first GPU
  try:
    tf.config.experimental.set_visible_devices(gpus[1], 'GPU')
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except RuntimeError as e:
    # Visible devices must be set before GPUs have been initialized
    print(e)

4 Physical GPUs, 1 Logical GPU


In [3]:
s25 = np.load('s25.npy')
h25 = np.load('h25.npy')

In [4]:
s25_labels = np.array([0 for _ in range(0,len(s25))])
h25_labels = np.array([1 for _ in range(0,len(h25))])

In [5]:
X = np.append(s25,h25,axis=0)

In [6]:
y = np.append(s25_labels,h25_labels,axis=0)

In [7]:
def evaluate_model(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 30, 16
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(LSTM(100, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01), input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation= 'selu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

In [8]:
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print( ' Accuracy: %.3f%% (+/-%.3f) ' % (m, s))

In [9]:
def run_experiment(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model(trainX, trainy, testX, testy)
        score = score * 100.0
        print( ' >#%d: %.3f ' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

In [10]:
run_experiment(X,y)

Epoch 1/30
58/58 [==============================] - 10s 177ms/step - loss: 3.2710 - accuracy: 0.5476
Epoch 2/30
58/58 [==============================] - 10s 177ms/step - loss: 2.2876 - accuracy: 0.6418
Epoch 3/30
58/58 [==============================] - 10s 177ms/step - loss: 1.8131 - accuracy: 0.7196
Epoch 4/30
58/58 [==============================] - 10s 178ms/step - loss: 1.5293 - accuracy: 0.7788
Epoch 5/30
58/58 [==============================] - 10s 177ms/step - loss: 1.4203 - accuracy: 0.7470
Epoch 6/30
58/58 [==============================] - 10s 178ms/step - loss: 1.2417 - accuracy: 0.8039
Epoch 7/30
58/58 [==============================] - 10s 178ms/step - loss: 1.1125 - accuracy: 0.8138
Epoch 8/30
58/58 [==============================] - 10s 179ms/step - loss: 1.0332 - accuracy: 0.8543
Epoch 9/30
58/58 [==============================] - 10s 177ms/step - loss: 0.9974 - accuracy: 0.8094
Epoch 10/30
58/58 [==============================] - 10s 177ms/step - loss: 1.1025 - accura

58/58 [==============================] - 10s 170ms/step - loss: 0.7255 - accuracy: 0.8478
Epoch 22/30
58/58 [==============================] - 10s 177ms/step - loss: 0.7099 - accuracy: 0.8368
Epoch 23/30
58/58 [==============================] - 10s 175ms/step - loss: 0.6936 - accuracy: 0.8456
Epoch 24/30
58/58 [==============================] - 10s 177ms/step - loss: 0.6790 - accuracy: 0.8576
Epoch 25/30
58/58 [==============================] - 10s 176ms/step - loss: 0.8037 - accuracy: 0.8215
Epoch 26/30
58/58 [==============================] - 10s 173ms/step - loss: 1.0318 - accuracy: 0.5969
Epoch 27/30
58/58 [==============================] - 10s 176ms/step - loss: 0.9729 - accuracy: 0.6331
Epoch 28/30
58/58 [==============================] - 10s 175ms/step - loss: 0.9524 - accuracy: 0.6068
Epoch 29/30
58/58 [==============================] - 10s 178ms/step - loss: 0.9431 - accuracy: 0.5980
Epoch 30/30
58/58 [==============================] - 10s 176ms/step - loss: 0.9200 - accuracy:

In [11]:
def evaluate_model_2(trainX, trainy, testX, testy):
    verbose, epochs, batch_size = 1, 30, 16
    n_timesteps, n_features = trainX.shape[1], trainX.shape[2]
    model = Sequential()
    model.add(LSTM(100, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01), return_sequences=True, input_shape=(n_timesteps,n_features)))
    model.add(LSTM(50, kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
    model.add(Dropout(0.5))
    model.add(Dense(100, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation= 'selu'))
    model.add(Dropout(0.25))
    model.add(Dense(1, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01), activation= 'sigmoid' ))
    model.compile(loss= 'binary_crossentropy' , optimizer= 'adam' , metrics=['accuracy'])
    # fit network
    model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
    return accuracy

In [12]:
def run_experiment_2(X,y,repeats=5):
    # load data
    trainX, testX,trainy, testy = train_test_split(X, y, test_size=0.20, random_state=42)
    mean = trainX.mean(axis=0)
    trainX -= mean
    std = trainX.std(axis=0)
    trainX /= std
    testX -= mean
    testX /= std
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model_2(trainX, trainy, testX, testy)
        score = score * 100.0
        print( ' >#%d: %.3f ' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

In [13]:
run_experiment_2(X,y)

Epoch 1/30
58/58 [==============================] - 17s 289ms/step - loss: 4.5514 - accuracy: 0.5641
Epoch 2/30
58/58 [==============================] - 17s 287ms/step - loss: 2.8324 - accuracy: 0.6780
Epoch 3/30
58/58 [==============================] - 17s 289ms/step - loss: 2.1496 - accuracy: 0.7459
Epoch 4/30
58/58 [==============================] - 17s 289ms/step - loss: 1.7911 - accuracy: 0.7831
Epoch 5/30
58/58 [==============================] - 17s 287ms/step - loss: 1.6217 - accuracy: 0.7798
Epoch 6/30
58/58 [==============================] - 17s 287ms/step - loss: 1.4612 - accuracy: 0.8007
Epoch 7/30
58/58 [==============================] - 17s 288ms/step - loss: 1.3003 - accuracy: 0.8149
Epoch 8/30
58/58 [==============================] - 17s 288ms/step - loss: 1.2232 - accuracy: 0.8138
Epoch 9/30
58/58 [==============================] - 17s 289ms/step - loss: 1.1865 - accuracy: 0.8039
Epoch 10/30
58/58 [==============================] - 17s 288ms/step - loss: 1.0478 - accura

58/58 [==============================] - 17s 288ms/step - loss: 0.6269 - accuracy: 0.8620
Epoch 22/30
58/58 [==============================] - 17s 290ms/step - loss: 0.6266 - accuracy: 0.8587
Epoch 23/30
58/58 [==============================] - 17s 290ms/step - loss: 0.5568 - accuracy: 0.8905
Epoch 24/30
58/58 [==============================] - 17s 289ms/step - loss: 0.8714 - accuracy: 0.6780
Epoch 25/30
58/58 [==============================] - 17s 294ms/step - loss: 0.7105 - accuracy: 0.7809
Epoch 26/30
58/58 [==============================] - 17s 289ms/step - loss: 0.5713 - accuracy: 0.8762
Epoch 27/30
58/58 [==============================] - 17s 290ms/step - loss: 0.5694 - accuracy: 0.8729
Epoch 28/30
58/58 [==============================] - 17s 290ms/step - loss: 0.5340 - accuracy: 0.8708
Epoch 29/30
58/58 [==============================] - 17s 287ms/step - loss: 0.5329 - accuracy: 0.8751
Epoch 30/30
58/58 [==============================] - 17s 289ms/step - loss: 0.5277 - accuracy: